In [1]:
import os
import sys
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import torch
from kan import KAN

from tqdm import tqdm

In [2]:
from raw_data_processing import get_x, get_y, get_wavelength
from tools import JSON_Read, plotly_multi_scatter, get_all_sqz_input, KAN_es

In [3]:
SCRIPT_DIR = os.path.abspath('')

## Loading data

In [4]:
d_config = JSON_Read("", "json_config.txt")

EXCITE_WAVE_LENGTH = d_config['EXCITE_WAVE_LENGTH']
PREDICT_IONS = d_config['PREDICT_IONS']
SPEC_FOLDER = d_config['SPEC_FOLDER']

TRAIN_TEST_RATIO = d_config['TRAIN_TEST_RATIO']
VALIDATION_TRAIN_RATIO = d_config['VALIDATION_TRAIN_RATIO']
N_ITER_NO_CHANGE = d_config['N_ITER_NO_CHANGE']

HIDDEN_LAYER_SIZES = d_config['HIDDEN_LAYER_SIZES']
ACTIVATION = d_config['ACTIVATION']
SOLVER = d_config['SOLVER']

In [5]:
x = get_x(wave_length=EXCITE_WAVE_LENGTH, spec_file=""+SPEC_FOLDER)
y = get_y(l_ions=PREDICT_IONS, spec_file=""+SPEC_FOLDER)

# Squeeze input data

In [6]:
l_wavelenth = get_wavelength(spec_file=""+SPEC_FOLDER)

In [7]:
x_matrix, y_matrix = np.broadcast_to(l_wavelenth, (len(x), len(l_wavelenth))), x.to_numpy()

x_sqz = get_all_sqz_input(x_matrix, y_matrix)

In [10]:
def alg_KAN_es(x, y, seed = None,
               K=3, GRID = 3,
               lamb=0., lamb_l1=1., lamb_entropy=2.,
               steps=200, tol=0.001, n_iter_no_change=10):
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        train_size=TRAIN_TEST_RATIO,
                                                        random_state=seed)

    x_val, x_train, y_val, y_train = train_test_split(x_train, y_train, 
                                                      train_size=VALIDATION_TRAIN_RATIO, 
                                                      random_state=seed)
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    x_test = scaler.transform(x_test)

    tc_x_train = torch.from_numpy(x_train)
    tc_y_train = torch.from_numpy(y_train.reshape([-1,1]))
    tc_x_val = torch.from_numpy(x_val)
    tc_y_val = torch.from_numpy(y_val).reshape([-1,1])
    tc_x_test = torch.from_numpy(x_test)
    tc_y_test = torch.from_numpy(y_test).reshape([-1,1])

    dataset_3 = {'train_input': tc_x_train,
                 'train_label': tc_y_train,
                 'val_input': tc_x_val,
                 'val_label': tc_y_val,
                 'test_input': tc_x_test,
                 'test_label': tc_y_test}
    
    INPUT_SHAPE = tc_x_test.shape[1]

    model_es = KAN_es(width=[INPUT_SHAPE, 1, 1], grid=GRID, k=K, seed=seed)
    result_es = model_es.train_es(dataset_3, 
                                  tol=tol, 
                                  n_iter_no_change=n_iter_no_change,
                                  opt="LBFGS", steps=steps, 
                                  lamb=lamb,
                                  lamb_l1=lamb_l1,
                                  lamb_entropy=lamb_entropy
                                  )
    
    pred_test = model_es(dataset_3['test_input']).cpu().detach().numpy().ravel()
    rmse = mean_squared_error(y_test, pred_test)
    r2 = r2_score(y_test, pred_test)
    mae = mean_absolute_error(y_test, pred_test)

    return [rmse, r2, mae]

In [8]:
def alg_skl_model(x, y, class_model, model_kwargs, seed = None):
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        train_size=TRAIN_TEST_RATIO,
                                                        random_state=seed)
    
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    #print(model_kwargs)
    model = class_model(random_state=seed, **model_kwargs)
    model.fit(x_train, y_train)

    pred_test = model.predict(x_test)
    rmse = mean_squared_error(y_test, pred_test)
    r2 = r2_score(y_test, pred_test)
    mae = mean_absolute_error(y_test, pred_test)

    return [rmse, r2, mae]

In [9]:
MLP_model_kwargs = {'hidden_layer_sizes': 16,
                  'activation': ACTIVATION,
                  'solver': SOLVER,
                  'early_stopping': True,
                  'validation_fraction': VALIDATION_TRAIN_RATIO,
                  'n_iter_no_change': N_ITER_NO_CHANGE,
                  'learning_rate_init': 0.001,
                  'learning_rate': 'adaptive'}

GB_model_kwargs = {'validation_fraction': VALIDATION_TRAIN_RATIO,
                   'n_iter_no_change': N_ITER_NO_CHANGE}

RF_model_kwargs = {}

In [11]:
def multi_exp(l_algos_names,
              l_algos,
              mult_X_Y,
              l_kwargs,
              l_metrics_names,
              num_iter):
    ''' Function, that process algos(X, Y) and returns df of their metrics. 
    '''
    res_list = []

    for alg, (x, y), kwargs, alg_name in zip(l_algos, mult_X_Y, l_kwargs, l_algos_names):
        print(f'--- Processing {alg_name}')

        for i in range(1, num_iter+1):
            print(f'iter: {i}')
            #print(kwargs)
            l_metrics = alg(x, y, seed=i, **kwargs)
            res_list.append([alg_name, i]+l_metrics)
        print('-------')

    return pd.DataFrame(res_list, columns=['alg_name', 'iter']+l_metrics_names)

In [15]:
l_algos_names=['500_KAN', '500_MLP', '500_RF', '500_GB',
               '5_KAN', '5_MLP', '5_RF', '5_GB']

l_algos=[alg_KAN_es, alg_skl_model, alg_skl_model, alg_skl_model,
         alg_KAN_es, alg_skl_model, alg_skl_model, alg_skl_model]

mult_X_Y=[(x, y), (x, y), (x, y), (x, y), 
          (x_sqz, y), (x_sqz, y), (x_sqz, y), (x_sqz, y)]

l_kwargs=[{},
          {'class_model': MLPRegressor,'model_kwargs': MLP_model_kwargs},
          {'class_model': RandomForestRegressor,'model_kwargs': RF_model_kwargs},
          {'class_model': GradientBoostingRegressor,'model_kwargs': GB_model_kwargs},
          {},
          {'class_model': MLPRegressor,'model_kwargs': MLP_model_kwargs},
          {'class_model': RandomForestRegressor,'model_kwargs': RF_model_kwargs},
          {'class_model': GradientBoostingRegressor,'model_kwargs': GB_model_kwargs},]

l_metrics_names=['rmse', 'r2', 'mae']

num_iter=25

In [16]:
'''
l_algos_names=['500_MLP', '500_RF', '500_GB',
               '5_MLP', '5_RF', '5_GB']

l_algos=[alg_skl_model, alg_skl_model, alg_skl_model,
         alg_skl_model, alg_skl_model, alg_skl_model]

mult_X_Y=[(x, y), (x, y), (x, y), 
          (x_sqz, y), (x_sqz, y), (x_sqz, y)]

l_kwargs=[{'class_model': MLPRegressor,'model_kwargs': MLP_model_kwargs},
          {'class_model': RandomForestRegressor,'model_kwargs': RF_model_kwargs},
          {'class_model': GradientBoostingRegressor,'model_kwargs': GB_model_kwargs},
          {'class_model': MLPRegressor,'model_kwargs': MLP_model_kwargs},
          {'class_model': RandomForestRegressor,'model_kwargs': RF_model_kwargs},
          {'class_model': GradientBoostingRegressor,'model_kwargs': GB_model_kwargs},]

l_metrics_names=['rmse', 'r2', 'mae']

num_iter=3
'''

"\nl_algos_names=['500_MLP', '500_RF', '500_GB',\n               '5_MLP', '5_RF', '5_GB']\n\nl_algos=[alg_skl_model, alg_skl_model, alg_skl_model,\n         alg_skl_model, alg_skl_model, alg_skl_model]\n\nmult_X_Y=[(x, y), (x, y), (x, y), \n          (x_sqz, y), (x_sqz, y), (x_sqz, y)]\n\nl_kwargs=[{'class_model': MLPRegressor,'model_kwargs': MLP_model_kwargs},\n          {'class_model': RandomForestRegressor,'model_kwargs': RF_model_kwargs},\n          {'class_model': GradientBoostingRegressor,'model_kwargs': GB_model_kwargs},\n          {'class_model': MLPRegressor,'model_kwargs': MLP_model_kwargs},\n          {'class_model': RandomForestRegressor,'model_kwargs': RF_model_kwargs},\n          {'class_model': GradientBoostingRegressor,'model_kwargs': GB_model_kwargs},]\n\nl_metrics_names=['rmse', 'r2', 'mae']\n\nnum_iter=3\n"

In [17]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_X_Y=mult_X_Y,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing 500_KAN
iter: 1


trn_ls: 1.60e-01 | vl_ls: 4.37e-01 | e_stop: 10/10 | tst_ls: 4.73e-01 | reg: 3.72e+01 :   7%|▎   | 14/200 [02:10<28:47,  9.29s/it]


Early stopping criteria raised
iter: 2


trn_ls: 1.38e-01 | vl_ls: 4.45e-01 | e_stop: 10/10 | tst_ls: 4.79e-01 | reg: 3.50e+01 :   7%|▎   | 14/200 [02:06<28:00,  9.04s/it]


Early stopping criteria raised
iter: 3


...

iter: 24


c:\Users\Gavriil\VS_projects\KAN_spectroscopy\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


iter: 25


c:\Users\Gavriil\VS_projects\KAN_spectroscopy\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


-------
--- Processing 5_RF
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
iter: 12
iter: 13
iter: 14
iter: 15
iter: 16
iter: 17
iter: 18
iter: 19
iter: 20
iter: 21
iter: 22
iter: 23
iter: 24
iter: 25
-------
--- Processing 5_GB
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
iter: 12
iter: 13
iter: 14
iter: 15
iter: 16
iter: 17
iter: 18
iter: 19
iter: 20
iter: 21
iter: 22
iter: 23
iter: 24
iter: 25
-------


In [18]:
full_df

,alg_name,iter,rmse,r2,mae
0,500_KAN,1,0.170453,0.923544,0.308077
1,500_KAN,2,0.188008,0.924614,0.315091
2,500_KAN,3,0.195467,0.921781,0.332320
3,500_KAN,4,0.132584,0.949010,0.281474
4,500_KAN,5,0.144570,0.939638,0.275084
...,...,...,...,...,...
195,5_GB,21,0.122070,0.951131,0.272301
196,5_GB,22,0.127524,0.951078,0.266599
197,5_GB,23,0.163647,0.932081,0.295245
198,5_GB,24,0.181779,0.918202,0.316071


In [19]:
full_df.to_excel('full_metrics.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [20]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics.xlsx')
aggr_df

rmse                  r2                 mae          
              mean       std      mean       std      mean       std
alg_name                                                            
500_GB    0.229024  0.035870  0.906359  0.015556  0.366623  0.028747
500_KAN   0.181186  0.029910  0.925875  0.013506  0.319783  0.028280
500_MLP   1.515338  0.394456  0.380907  0.166678  0.981519  0.123861
500_RF    0.223663  0.038748  0.908481  0.017274  0.355509  0.027921
5_GB      0.169189  0.032629  0.930912  0.013644  0.300406  0.019115
5_KAN     0.118371  0.024763  0.951499  0.011249  0.251473  0.025528
5_MLP     0.256118  0.038250  0.895525  0.015454  0.403326  0.035079
5_RF      0.188030  0.041634  0.923181  0.017504  0.292365  0.024603